In [10]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

In [11]:
dict_rxn_to_gene["METLEUex"]

['6520.1', '8140.1']

In [12]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/data_for_R/Non_masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)


edges_node_names_explainer_subgraphs

,node1,node2,0,1,2,3,4,5,6,7,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,OIVD2m,coa_m,0.011614,0.012127,0.015279,0.017670,0.011225,0.021512,0.010746,0.016253,...,0.013741,0.014514,0.016654,0.041726,0.012164,0.018715,0.013917,0.025895,0.015294,0.011570
1,r0308,coa_x,0.012449,0.011110,0.015181,0.018134,0.011879,0.019576,0.011392,0.015874,...,0.013629,0.014181,0.015770,0.015426,0.010993,0.017803,0.036254,0.021730,0.014780,0.012422
2,r1571,ala_L_e,0.010841,0.011814,0.016486,0.019439,0.011499,0.022255,0.011188,0.016937,...,0.015688,0.014428,0.015809,0.023927,0.010487,0.021306,0.014627,0.024803,0.014350,0.010247
3,r1571,ala_L_c,0.009146,0.010015,0.019487,0.020649,0.009553,0.024320,0.009470,0.017769,...,0.007124,0.014661,0.016414,0.027540,0.008362,0.022674,0.014731,0.027482,0.015170,0.008430
4,CRNt,crn_c,0.011504,0.011552,0.015367,0.017667,0.011123,0.020875,0.011553,0.016309,...,0.013385,0.014350,0.015244,0.014297,0.010706,0.019663,0.024745,0.021713,0.014034,0.010986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,RE0938C,phe_L_c,0.013603,0.013524,0.014519,0.015714,0.013481,0.014389,0.013471,0.017052,...,0.013632,0.020183,0.018640,0.015064,0.013631,0.014972,0.016736,0.014202,0.018839,0.013591
386,RE0938C,CE2917_c,0.013392,0.013574,0.014433,0.015266,0.013639,0.014400,0.013615,0.016391,...,0.013613,0.018718,0.017580,0.014811,0.013511,0.014532,0.015740,0.013800,0.017897,0.013566
387,r1783,phe_L_e,0.013417,0.013381,0.010611,0.016075,0.013382,0.014011,0.013419,0.015938,...,0.013410,0.017229,0.018466,0.016221,0.013532,0.014518,0.034531,0.013943,0.015708,0.013463
388,r1546,gly_e,0.016928,0.016705,0.008758,0.008303,0.016559,0.009065,0.016995,0.010558,...,0.019154,0.013087,0.011154,0.014383,0.017582,0.007615,0.014448,0.010110,0.012198,0.017527


In [13]:

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]


2022-10-27 23:46:57,651	INFO worker.py:1518 -- Started a local Ray instance.


In [14]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,"[1738.1, 1629.1, 594.1, 593.1, 594.2]",OIVD2m,coa_m,0.011614,0.012127,0.015279,0.017670,0.011225,0.021512,0.010746,...,0.013741,0.014514,0.016654,0.041726,0.012164,0.018715,0.013917,0.025895,0.015294,0.011570
1,"[10965.1, 122970.1, 11332.1, 5538.1, 641371.1,...",r0308,coa_x,0.012449,0.011110,0.015181,0.018134,0.011879,0.019576,0.011392,...,0.013629,0.014181,0.015770,0.015426,0.010993,0.017803,0.036254,0.021730,0.014780,0.012422
2,[8140.1],r1571,ala_L_e,0.010841,0.011814,0.016486,0.019439,0.011499,0.022255,0.011188,...,0.015688,0.014428,0.015809,0.023927,0.010487,0.021306,0.014627,0.024803,0.014350,0.010247
3,[8140.1],r1571,ala_L_c,0.009146,0.010015,0.019487,0.020649,0.009553,0.024320,0.009470,...,0.007124,0.014661,0.016414,0.027540,0.008362,0.022674,0.014731,0.027482,0.015170,0.008430
4,[6584.1],CRNt,crn_c,0.011504,0.011552,0.015367,0.017667,0.011123,0.020875,0.011553,...,0.013385,0.014350,0.015244,0.014297,0.010706,0.019663,0.024745,0.021713,0.014034,0.010986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,[59272.1],RE0938C,phe_L_c,0.013603,0.013524,0.014519,0.015714,0.013481,0.014389,0.013471,...,0.013632,0.020183,0.018640,0.015064,0.013631,0.014972,0.016736,0.014202,0.018839,0.013591
386,[59272.1],RE0938C,CE2917_c,0.013392,0.013574,0.014433,0.015266,0.013639,0.014400,0.013615,...,0.013613,0.018718,0.017580,0.014811,0.013511,0.014532,0.015740,0.013800,0.017897,0.013566
387,[9057.1],r1783,phe_L_e,0.013417,0.013381,0.010611,0.016075,0.013382,0.014011,0.013419,...,0.013410,0.017229,0.018466,0.016221,0.013532,0.014518,0.034531,0.013943,0.015708,0.013463
388,[8140.1],r1546,gly_e,0.016928,0.016705,0.008758,0.008303,0.016559,0.009065,0.016995,...,0.019154,0.013087,0.011154,0.014383,0.017582,0.007615,0.014448,0.010110,0.012198,0.017527


In [15]:
results_for_R.to_csv("./results/data/data_for_R/Non_masked_results_for_R.csv")

# Masked

In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo
    
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/data_for_R/Masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)
ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]



Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


2022-10-28 22:11:53,438	INFO worker.py:1518 -- Started a local Ray instance.


In [2]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R.query("node1 == 'METLEUex'")

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
228,"[6520.1, 8140.1]",METLEUex,leu_L_c,0.015146,0.018446,0.07101,0.242309,0.351374,0.08717,0.244823,...,0.099365,0.371835,0.096621,0.015923,0.265067,0.456124,0.014109,0.017292,0.194261,0.016662
229,"[6520.1, 8140.1]",METLEUex,leu_L_e,0.015889,0.014949,0.60489,0.615560,0.620651,0.77503,0.776815,...,0.604178,0.650645,0.654546,0.017878,0.745853,0.816301,0.016309,0.014953,0.875668,0.656670


In [3]:
results_for_R.to_csv("./results/data/data_for_R/Masked_results_for_R.csv")